## Day 24: Immune System Simulator 20XX

https://adventofcode.com/2018/day/24

### Part 1

This looks tedious and frustrating but I'm not giving up this close to the end.

In [84]:
from parse import parse
from collections import namedtuple
from pyrsistent import pmap, pset


Group = namedtuple('Group', 'id type units hit_points immune weak attack_points damage_type initiative')


def parse_system(data):
    groups = pmap()
    
    lines = [line.strip() for line in data.strip().splitlines()]
    
    type_parsing = 'immune'
    group_id = 0
    
    parse_string = '{n:d} units each with {hp:d} hit points{immune_weak}with an attack that does {ap:d} {damage_type} damage at initiative {initiative:d}'
    
    for line in lines[1:]:
        if line == 'Infection:':
            type_parsing = 'infection'
        else:
            p = parse(parse_string, line)
            
            if p:
                weak = pset()
                immune = pset()
                iws = p['immune_weak'].strip().strip('()').split(';')

                for iw in iws:
                    if iw:
                        words = iw.split()
                        damage_types = pset([s.strip(',') for s in words[2:]])
                        if words[0] == 'immune':
                            immune = damage_types
                        else:
                            weak = damage_types
                        
                groups = groups.set(group_id, 
                                    Group(group_id, type_parsing, p['n'], p['hp'], 
                                          immune, weak, p['ap'], p['damage_type'], 
                                          p['initiative']))
                group_id += 1

    return groups

In [85]:
test_data = '''Immune System:
17 units each with 5390 hit points (weak to radiation, bludgeoning) with an attack that does 4507 fire damage at initiative 2
989 units each with 1274 hit points (immune to fire; weak to bludgeoning, slashing) with an attack that does 25 slashing damage at initiative 3

Infection:
801 units each with 4706 hit points (weak to radiation) with an attack that does 116 bludgeoning damage at initiative 1
4485 units each with 2961 hit points (immune to radiation; weak to fire, cold) with an attack that does 12 slashing damage at initiative 4'''

test_groups = parse_system(test_data)
test_groups

pmap({0: Group(id=0, type='immune', units=17, hit_points=5390, immune=pset(), weak=pset(['bludgeoning', 'radiation']), attack_points=4507, damage_type='fire', initiative=2), 1: Group(id=1, type='immune', units=989, hit_points=1274, immune=pset(['fire']), weak=pset(['slashing', 'bludgeoning']), attack_points=25, damage_type='slashing', initiative=3), 2: Group(id=2, type='infection', units=801, hit_points=4706, immune=pset(), weak=pset(['radiation']), attack_points=116, damage_type='bludgeoning', initiative=1), 3: Group(id=3, type='infection', units=4485, hit_points=2961, immune=pset(['radiation']), weak=pset(['cold', 'fire']), attack_points=12, damage_type='slashing', initiative=4)})

In [133]:
def damage(attacker, defender):
    if attacker.damage_type in defender.immune:
        return 0
    elif attacker.damage_type in defender.weak:
        return 2 * attacker.units * attacker.attack_points
    else: 
        return attacker.units * attacker.attack_points
    
    
def target_selection(groups, debug=False):
    targets = pmap()
    
    attackers = sorted(groups,
                       key = lambda g: (groups[g].units * groups[g].attack_points,
                                        groups[g].initiative),
                       reverse=True)
    attacked = pset()
    
    for attack_id in attackers:
        attacker = groups[attack_id]
        
        defenders = [groups[d] 
                     for d in groups 
                     if groups[d].type != attacker.type 
                     and groups[d].id not in attacked]
        
        if defenders:
            if debug:
                for defender in sorted(defenders, key=lambda d: d.id):
                    # This is clumsy, for debugging
                    d = damage(attacker, defender)
                t = 'Immune System' if attacker.type == 'immune' else 'Infection'
                print(f'{t} group {attacker.id} would deal defending group {defender.id} {d} damage')

            selected = max(defenders, 
                           key=lambda d: (damage(attacker, d),
                                          d.units * d.attack_points,
                                          d.initiative))
            if damage(attacker, selected) > 0:
                targets = targets.set(attack_id, pset([selected.id]))
                attacked = attacked.add(selected.id)
            else:
                targets = targets.set(attack_id, pset())
        else: 
            targets = targets.set(attack_id, pset())
    
    return targets

In [134]:
target_selection(test_groups, debug=True)

Infection group 2 would deal defending group 1 185832 damage
Immune System group 0 would deal defending group 3 153238 damage
Infection group 3 would deal defending group 1 107640 damage
Immune System group 1 would deal defending group 2 24725 damage


pmap({0: pset([3]), 1: pset([2]), 2: pset([0]), 3: pset([1])})

In [135]:
def attack(groups, attack_id, defend_id, debug=False):
    if attack_id in groups and defend_id in groups:
        attacker = groups[attack_id]
        defender = groups[defend_id]

        d = damage(attacker, defender)
        kills = min(d // defender.hit_points, defender.units)
        defender = defender._replace(units = defender.units - kills)
        
        if debug:
            att_t = 'Immune System' if attacker.type == 'immune' else 'Infection'
            print(f'{att_t} group {attack_id} attacks defending group {defend_id}, killing {kills} units')
        if defender.units > 0:
            return groups.set(defend_id, defender)
        else:
            return groups.discard(defend_id)
    else:
        return groups
    

def fight(groups, debug=False):
    targets = target_selection(groups, debug)
    
    if debug:
        print()

    for attack_id in sorted(targets,
                            key=lambda g: groups[g].initiative,
                            reverse=True):
        for defend_id in targets[attack_id]:
            groups = attack(groups, attack_id, defend_id, debug)
        
    return groups
        
        
fight(test_groups, debug=True)

Infection group 2 would deal defending group 1 185832 damage
Immune System group 0 would deal defending group 3 153238 damage
Infection group 3 would deal defending group 1 107640 damage
Immune System group 1 would deal defending group 2 24725 damage

Infection group 3 attacks defending group 1, killing 84 units
Immune System group 1 attacks defending group 2, killing 4 units
Immune System group 0 attacks defending group 3, killing 51 units
Infection group 2 attacks defending group 0, killing 17 units


pmap({1: Group(id=1, type='immune', units=905, hit_points=1274, immune=pset(['fire']), weak=pset(['slashing', 'bludgeoning']), attack_points=25, damage_type='slashing', initiative=3), 2: Group(id=2, type='infection', units=797, hit_points=4706, immune=pset(), weak=pset(['radiation']), attack_points=116, damage_type='bludgeoning', initiative=1), 3: Group(id=3, type='infection', units=4434, hit_points=2961, immune=pset(['radiation']), weak=pset(['cold', 'fire']), attack_points=12, damage_type='slashing', initiative=4)})

In [136]:
def print_groups(groups):
    print('Immune System:')
    imm_ids = [i for i in groups if groups[i].type == 'immune']
    if imm_ids:
        for i in sorted(imm_ids):
            print(f'Group {i} contains {groups[i].units} units')
    else:
        print('No groups remain')
        
    print('Infection:')
    inf_ids = [i for i in groups if groups[i].type == 'infection']
    if inf_ids:
        for i in sorted(inf_ids):
            print(f'Group {i} contains {groups[i].units} units')
    else:
        print('No groups remain') 
    

def battle(groups, debug=False):
    while any(True for g in groups if groups[g].type == 'immune') and \
        any(True for g in groups if groups[g].type == 'infection'):
        if debug:
            print_groups(groups)
            print()
        groups = fight(groups, debug)
        
    if debug:
        print()
        print_groups(groups)
        
    return sum(groups[g].units for g in groups)

In [137]:
battle(test_groups, debug=True)

Immune System:
Group 0 contains 17 units
Group 1 contains 989 units
Infection:
Group 2 contains 801 units
Group 3 contains 4485 units

Infection group 2 would deal defending group 1 185832 damage
Immune System group 0 would deal defending group 3 153238 damage
Infection group 3 would deal defending group 1 107640 damage
Immune System group 1 would deal defending group 2 24725 damage

Infection group 3 attacks defending group 1, killing 84 units
Immune System group 1 attacks defending group 2, killing 4 units
Immune System group 0 attacks defending group 3, killing 51 units
Infection group 2 attacks defending group 0, killing 17 units
Immune System:
Group 1 contains 905 units
Infection:
Group 2 contains 797 units
Group 3 contains 4434 units

Infection group 2 would deal defending group 1 184904 damage
Immune System group 1 would deal defending group 3 22625 damage

Immune System group 1 attacks defending group 2, killing 4 units
Infection group 2 attacks defending group 1, killing 144 u

5216

It works for the test data, now to have my day ruined by edge cases on the problem data. 

In [147]:
groups = parse_system(open('input', 'r').read())
%time battle(groups)

CPU times: user 780 ms, sys: 0 ns, total: 780 ms
Wall time: 780 ms


33551

It did, I wasn't tie-breaking when selecting which opponent to attack, but that was spotted thankfully quickly.

### Part 2

In [139]:
def boost(groups, n):
    for g in groups:
        if groups[g].type == 'immune':
            groups = groups.set(g, 
                                groups[g]._replace(attack_points = groups[g].attack_points + n))
            
    return groups
            
boost(test_groups, 1570)

pmap({0: Group(id=0, type='immune', units=17, hit_points=5390, immune=pset(), weak=pset(['bludgeoning', 'radiation']), attack_points=6077, damage_type='fire', initiative=2), 1: Group(id=1, type='immune', units=989, hit_points=1274, immune=pset(['fire']), weak=pset(['slashing', 'bludgeoning']), attack_points=1595, damage_type='slashing', initiative=3), 2: Group(id=2, type='infection', units=801, hit_points=4706, immune=pset(), weak=pset(['radiation']), attack_points=116, damage_type='bludgeoning', initiative=1), 3: Group(id=3, type='infection', units=4485, hit_points=2961, immune=pset(['radiation']), weak=pset(['cold', 'fire']), attack_points=12, damage_type='slashing', initiative=4)})

In [145]:
def battle_winner(groups, debug=False):
    while any(True for g in groups if groups[g].type == 'immune') and \
        any(True for g in groups if groups[g].type == 'infection'):
        if debug:
            print_groups(groups)
            print()
        old_groups = groups
        groups = fight(groups, debug)
        if groups == old_groups:
            return 'draw', groups
        
    if debug:
        print()
        print_groups(groups)
        
    if any(True for g in groups if groups[g].type == 'immune'):
        return 'immune', groups
    else:
        return 'infection', groups
    

battle_winner(boost(test_groups, 1570), debug=True)

Immune System:
Group 0 contains 17 units
Group 1 contains 989 units
Infection:
Group 2 contains 801 units
Group 3 contains 4485 units

Immune System group 1 would deal defending group 3 1577455 damage
Immune System group 0 would deal defending group 3 206618 damage
Infection group 2 would deal defending group 1 185832 damage
Infection group 3 would deal defending group 0 53820 damage

Infection group 3 attacks defending group 0, killing 9 units
Immune System group 1 attacks defending group 2, killing 335 units
Immune System group 0 attacks defending group 3, killing 32 units
Infection group 2 attacks defending group 1, killing 84 units
Immune System:
Group 0 contains 8 units
Group 1 contains 905 units
Infection:
Group 2 contains 466 units
Group 3 contains 4453 units

Immune System group 1 would deal defending group 3 1443475 damage
Infection group 2 would deal defending group 1 108112 damage
Infection group 3 would deal defending group 0 53436 damage
Immune System group 0 would deal de

('immune',
 pmap({1: Group(id=1, type='immune', units=51, hit_points=1274, immune=pset(['fire']), weak=pset(['slashing', 'bludgeoning']), attack_points=1595, damage_type='slashing', initiative=3)}))

A binary search would be a good idea here but I'm lazy so will iterate by hand.

In [148]:
stopped = False
n = 0
while not stopped:
    winner, remainder = battle_winner(boost(groups, n))
    if winner == 'immune':
        print(n)
        stopped = True
    n += 1000

1000


In [149]:
stopped = False
n = 0
while not stopped:
    winner, remainder = battle_winner(boost(groups, n))
    if winner == 'immune':
        print(n)
        stopped = True
    n += 100

100


In [150]:
stopped = False
n = 0
while not stopped:
    winner, remainder = battle_winner(boost(groups, n))
    if winner == 'immune':
        print(n)
        stopped = True
    n += 10

80


In [151]:
stopped = False
n = 61
while not stopped:
    winner, remainder = battle_winner(boost(groups, n))
    if winner == 'immune':
        print(n)
        stopped = True
    n += 1

77


In [152]:
sum(g.units for g in remainder.values())

760

I am not proud of this.